## [**Esempio**](http://image.made-in-china.com/43f34j00ejnErSaPgLqY/High-Rise-Prefabricated-Steel-Structure-Building-S-S-051-.jpg)

In [1]:
"""Creates simple parametric building structure."""

from larlib import *

Evaluating fenvs.py..
...fenvs.py imported in 0.0064199399294 seconds


C:\Program Files (x86)\Anaconda2\lib\site-packages\larlib-0.3.2-py2.7.egg\larlib\larstruct.py:233: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  self.body = [item for item in data if item != None]


In [2]:
def buildSpaceFrame(px,py,bx,bz,dy,hz):
	"""Creates and shows a space frame of reinforced concrete."""
	pillars = buildPillars(px,py,bz,dy,hz)
	beams = buildBeams(px,bx,bz,dy,hz)
	pillars=COLOR(RED)(pillars)
	beams=COLOR(BLUE)(beams)
	spaceFrame=STRUCT([pillars,beams])
	return spaceFrame


def buildPillars(px,py,bz,dy,hz):
	"""Creates pillars of the building's space frame."""
	x = [px]
	for i in dy:
		x.append(-i)
		x.append(px)
	y = [py]
	bases = PROD([QUOTE(x),QUOTE(y)])
	l=0
	for j in hz:
		pillars[l]=PROD([bases,QUOTE([j])])
		l=l+1
	z=hz[0]
	allPillars=STRUCT([pillars[0],T(3)(z+bz),pillars[1]])
	for i in range(1,len(hz)-1):
		z=z+hz[i]
		allPillars=STRUCT([allPillars,T(3)([z+bz*(i+1)]),pillars[i+1]])
	return allPillars

def buildBeams(px,bx,bz,dy,hz):
	"""Creates beams of the building's space frame."""
	x=[]
	for i in dy:
		x.append(i+px*2)
	y=[bx]
	bases=PROD([QUOTE(x),QUOTE(y)])
	beams=PROD([bases,QUOTE([bz])])
	beams=T(1)(-px/2)(beams)
	beams=T(3)(hz[0])(beams)
	z=0
	allBeams=beams
	for i in range(1,len(hz)):
		z=z+hz[i]
		allBeams=STRUCT([allBeams,T(3)(z+bz*i),beams])
	return allBeams
		

In [5]:
def ggpl_bone_structure(file_name):
    """Reads parameters from file_name and shows the bone structure of a building."""
    import csv
    ifile  = open(file_name, "rb")
    reader = csv.reader(ifile,delimiter=';')
    frameSizes = []
    vectors = []
    frames = []
    connBeams = []
    n=0
    startPointY=0
    for row in reader:
        v=[]
        for j in row[:3]:
            v.append(eval(j))
        vectors.append(v)
        evenRow=next(reader)
        frameSizes.append(evenRow)
    for params in frameSizes:
        dy=[]
        hz=[]
        y=params[4].split()
        z=params[5].split()
        for d in y:
            dy.append(eval(d))
        for h in z:
            hz.append(eval(h))
        px=eval(params[0])
        py=eval(params[1])
        bx=eval(params[2])
        bz=eval(params[3])
        a = buildSpaceFrame(px,py,bx,bz,dy,hz)
        frames.append(a)
        if n<len(vectors)-1:
            startPointY=startPointY+vectors[n][1]
            b = buildConnectingBeams(px,py,bx,bz,dy,hz,vectors[n][0],startPointY,vectors[n+1][1])
            connBeams.append(b)
        n=n+1
    allConnBeams = connBeams[0]
    for k in range(1,len(connBeams)):
        allConnBeams=STRUCT([allConnBeams,connBeams[k]])
    structure = T([1,2,3])(vectors[0])(frames[0])
    d=vectors[0]
    for i in range(1,len(vectors)):
        d[0]=d[0]+vectors[i][0]
        d[1]=d[1]+vectors[i][1]
        d[2]=d[2]+vectors[i][2]
        structure = STRUCT([structure,T([1,2,3])([d[0],d[1],d[2]]),frames[i]])
    structure = STRUCT([structure,allConnBeams])
    VIEW(structure)
    return structure
    
def buildConnectingBeams(px,py,bx,bz,dy,hz,startPointX,startPointY,frameDistance):
    """Creates the beams connecting two frames"""
    x = [px]
    for i in dy:
        x.append(-i)
        x.append(px)    
    bases = PROD([QUOTE(x),QUOTE([frameDistance-py])])
    beams = T([1,2,3])([startPointX,startPointY+py,hz[0]])(PROD([bases,QUOTE([bz])]))
    z=0
    allBeams=beams
    for i in range(1,len(hz)):
        z=z+hz[i]
        allBeams = STRUCT([allBeams,T(3)(bz*i+z),beams])
    return allBeams

In [7]:
ggpl_bone_structure('frame_data_445315.csv')

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x03AD6518> >